### Variation 1: Using Lists

In [ ]:
from rich.prompt import Prompt
from rich.console import Console
from random import choice
import random
import typing
import sys

SQUARES = {
    'correct_place': '🟩',
    'correct_letter': '🟨',
    'incorrect_letter': '⬛'
}

GAMEWORD_LIST_FNAME = "gamewords.txt"
GUESSWORD_LIST_FNAME = "guesswords.txt"


def filter_word(word: str, length: int) -> typing.Optional[str]:
#Filter a word to add it to the wordlist
    if len(word.strip()) == length:
        return word.strip().upper()
    return None


def create_wordlist(fname: str, length: int) -> typing.List[str]:
#create a wordlist
    with open(fname, "r") as f:
        lines = f.readlines()

    # Filter out none values before creating the word list
    return [word for word in map(lambda word: filter_word(word, length), lines) if word is not None]


def validate(guess: str, wordlen: int, wordlist: typing.List[str]) -> typing.Tuple[str, str]:
#Validate a guess from a user.
# make sure the guess is all upper case to create uniformity with wordlist
    guess_upper = guess.upper()
    if guess_upper == 'Q':
            print(f"You quit - the correct answer was {WORD.upper()} and you took {NUM_GUESSES} guesses")
            return None, guess_upper

    if len(guess_upper) != wordlen:
        return f"Guess must be of length {wordlen}", guess_upper

    # guesses must also be words from the word list
    if guess_upper not in wordlist:
        return "Guess must be a valid word", guess_upper
    return None, guess_upper


def get_user_guess(wordlen: int, wordlist: typing.List[str]) -> str:
#get input,validate and return
    # continue looping until you get a valid guess
    while True:
        guess = Prompt.ask("Guess:")
        error, guess = validate(guess=guess, wordlen=wordlen,
                                wordlist=wordlist)
        if error is None:
            break

#show the input error 
        print(error)
    return guess


def find_all_char_positions(word: str, char: str) -> typing.List[int]:
#find all the indices of that character
    positions = []
    pos = word.find(char)
    while pos != -1:
        positions.append(pos)
        pos = word.find(char, pos + 1)
    return positions


def compare(expected: str, guess: str) -> typing.List[str]:
#Compare the guess with the expected word 
# the output is incorrect to start,
# and as we progress through the checking, update each position in output list
    
    output = [SQUARES['incorrect_letter']] * len(expected)
    counted_pos = []

#check for correct words in the correct positions
#update output
    for index, (expected_char, guess_char) in enumerate(zip(expected, guess)):
        if expected_char == guess_char:
#correct character in the correct position
            output[index] = SQUARES['correct_place']
            counted_pos.append(index)

# now we check for the remaining letters that are in incorrect
    for index, guess_char in enumerate(guess):
        
        if guess_char in expected and \
                index not in counted_pos:
#all the positions the guessed
# character is present or not
            positions = find_all_char_positions(word=expected, char=guess_char)
            # have we accounted for all the positions
            for pos in positions:
                # if we have not accounted for the correct
                if pos not in counted_pos:
                    output[index] = SQUARES['correct_letter']
                    counted_pos.append(pos)
                    # we only count the "correct letter" once,

                    break
    #return the list
    return output


if __name__ == '__main__':
# the game word is 5 letters
    WORDLEN = 5
 #load the wordlist
    GAMEWORD_WORDLIST = create_wordlist(
        GAMEWORD_LIST_FNAME, length=WORDLEN)

# load the wordlist for the guesses
    GUESSWORD_WORDLIST = create_wordlist(
        GUESSWORD_LIST_FNAME, length=WORDLEN)

# set the maximum number of guesses
    MAX_GUESSES = 6

    continue_game = True  # Flag to control whether to continue the game

    while continue_game:  # Start a new loop
        # Reset the number of guesses for each new game
        NUM_GUESSES = 0

#select a random word to start with
        WORD = random.choice(GAMEWORD_WORDLIST)
        GAME_WORD_LENGTH = len(WORD)

#game instructions
        console = Console()
        console.print("""
Guess 5 letter words one at a time to guess the game word.
    
    🟩 character means a letter was guessed correctly
    in the correct position.
    🟨 character means a letter was guessed correctly,
    but in the incorrect position.
    ⬛ Incorrect letter.
    
    To quit, enter 'q'.
""")

        console.print(" ".join([SQUARES['incorrect_letter']] * GAME_WORD_LENGTH))

        try:
            while NUM_GUESSES < MAX_GUESSES:
#get a user guess
                GUESS = get_user_guess(
                    wordlen=GAME_WORD_LENGTH, wordlist=GUESSWORD_WORDLIST)

                if GUESS == 'Q':  # quitting situation
                    user_decision = input("Do you want to play again? (y/n): ").strip().lower()
                    if user_decision.lower() == 'y':
                        #Start a new game
                        WORD = random.choice(GAMEWORD_WORDLIST)
                        GAME_WORD_LENGTH = len(WORD)
                        NUM_GUESSES = 0
                        console.print(" ".join([SQUARES['incorrect_letter']] * GAME_WORD_LENGTH))
                        continue
                    else:
                        continue_game = False  #Exit the game loop
                        break
                else:
                    NUM_GUESSES += 1

                # display the guess 
                result = compare(expected=WORD, guess=GUESS)
                console.print(" ".join(result))

                if WORD == GUESS:
                    console.print(f"You won! It took you {NUM_GUESSES} guesses.")
                    break

            if NUM_GUESSES == MAX_GUESSES:
                console.print(f"Sorry, you reached the maximum number of guesses. The correct answer was {WORD.upper()}.")

        except KeyboardInterrupt:
            console.print(f"""
You quit - the correct answer was {WORD.upper()}
and you took {NUM_GUESSES} guesses
""")

        if continue_game:
            # Prompt the user to continue the game
            continue_game = input("Do you want to play again? (y/n): ").strip().lower() == 'y'  
    print("Thanks for playing!")  

Guess 5 letter words one at a time to guess the game word.
    
    🟩 character means a letter was guessed correctly
    in the correct position.
    🟨 character means a letter was guessed correctly,
    but in the incorrect position.
    ⬛ Incorrect letter.
    
    To quit, enter 'q'.

⬛ ⬛ ⬛ ⬛ ⬛

Guess::

water


⬛ 🟨 ⬛ ⬛ ⬛

...

amino


🟩 ⬛ 🟩 🟩 ⬛

Guess::

aling
Guess must be a valid word


Guess::

Wordlist Creation (create_wordlist):
Time Complexity: O(n * m), where n is the number of words in the file and m is the average length of each word.

Validation (validate function):
Time Complexity: O(n), where n is the length of the wordlist. 

User Input and Guess Comparison (get_user_guess and compare functions):
Both functions involve iterating over the characters of the guessed word and the expected word. The length of the words (WORDLEN) determines the maximum iterations. Thus, the time complexity is O(WORDLEN).

Main Loop:
The main loop iterates for a maximum of MAX_GUESSES times. Within each iteration, user input is taken and compared, so the time complexity of the loop is O(MAX_GUESSES * WORDLEN).

Overall, the time complexity of the program is dominated by the main loop, making it O(MAX_GUESSES * WORDLEN).


Space Complexity: 
O(n * m), where n is the number of words in the file and m is the average length of each word. Both lists store all the words from the corresponding files.
The space complexity for other variables is minimal and can be considered constant, not affecting the overall space complexity significantly.

### Variation 2: Using Sets

In [ ]:
from rich.prompt import Prompt
from rich.console import Console
import random
import typing

SQUARES = {
    'correct_place': '🟩',
    'correct_letter': '🟨',
    'incorrect_letter': '⬛'
}

GAMEWORD_LIST_FNAME = "gamewords.txt"
GUESSWORD_LIST_FNAME = "guesswords.txt"

WORD = ""  

def filter_word(word: str, length: int) -> typing.Optional[str]:
    return word.strip().upper() if len(word.strip()) == length else None

def create_wordset(fname: str, length: int) -> typing.Set[str]:
    with open(fname, "r") as f:
        lines = f.readlines()
    return {word for word in map(lambda word: filter_word(word, length), lines) if word}

def get_user_guess(wordlen: int, wordset: typing.Set[str]) -> str:
    while True:
        guess = Prompt.ask("Guess:")
        guess_upper = guess.upper()
        if guess_upper == 'Q':
            return guess_upper
        if len(guess_upper) != wordlen or guess_upper not in wordset:
            print(f"Guess must be of length {wordlen} and a valid word")
            continue
        return guess_upper

def find_all_char_positions(word: str, char: str) -> typing.Set[int]:
    positions = set()
    pos = word.find(char)
    while pos != -1:
        positions.add(pos)
        pos = word.find(char, pos + 1)
    return positions

def compare(expected: str, guess: str) -> typing.List[str]:
    output = [SQUARES['incorrect_letter']] * len(expected)
    counted_pos = set()

    for index, (expected_char, guess_char) in enumerate(zip(expected, guess)):
        if expected_char == guess_char:
            output[index] = SQUARES['correct_place']
            counted_pos.add(index)

    for index, guess_char in enumerate(guess):
        if guess_char in expected and output[index] == SQUARES['incorrect_letter']:
            positions = find_all_char_positions(word=expected, char=guess_char)
            for pos in positions:
                if pos not in counted_pos:
                    output[index] = SQUARES['correct_letter']
                    counted_pos.add(pos)
                    break
    return output

def main():
    global WORD
    WORDLEN = 5
    GAMEWORD_WORDSET = create_wordset(GAMEWORD_LIST_FNAME, length=WORDLEN)
    WORD = random.choice(list(GAMEWORD_WORDSET))
    GAME_WORD_LENGTH = len(WORD)
    MAX_GUESSES = 6
    NUM_GUESSES = 0

    console = Console()
    console.print("""
    Guess words one at a time to guess the game word.
    
    🟩 character means a letter was guessed correctly
    in the correct position.
    🟨 character means a letter was guessed correctly,
    but in the incorrect position.
    ⬛ Incorrect letter.
    
    To quit, enter 'q'.
    """)

    console.print(" ".join(['⬛'] * GAME_WORD_LENGTH))

    try:
        while NUM_GUESSES < MAX_GUESSES:
            GUESS = get_user_guess(wordlen=GAME_WORD_LENGTH, wordset=GAMEWORD_WORDSET)
            if GUESS == 'Q':
                console.print(f"You quit - the correct answer was {WORD.upper()} and you took {NUM_GUESSES} guesses")
                break
            NUM_GUESSES += 1
            result = compare(expected=WORD, guess=GUESS)
            console.print(" ".join(result))

            if WORD == GUESS:
                console.print(f"You won! It took you {NUM_GUESSES} guesses.")
                break

        if NUM_GUESSES == MAX_GUESSES:
            console.print(f"Sorry, you reached the maximum number of guesses. The correct answer was {WORD.upper()}.")

    except KeyboardInterrupt:
        console.print(f"You quit - the correct answer was {WORD.upper()} and you took {NUM_GUESSES} guesses")

    user_decision = input("Do you want to play again? (y/n): ").strip().lower()
    if user_decision == 'y':
        main() 
    else:
        console.print("Thanks for playing!")

if __name__ == '__main__':
    main()


create_wordset:
The time complexity is approximately O(n * m),where n is the number of words in the file and m is the average length of the words(as each word needs to be processed to check its length and then added to the set).

get_user_guess:Let n be the number of words in the wordset. The time complexity is O(n).

find_all_char_positions: Let n be the length of the word. The time complexity is O(n).

compare: Let n be the length of the word. The time complexity is O(n).

main function: Let m be the maximum number of guesses and n is the number of words in the file. Overall, the time complexity is O(n * m).

Space Complexity: O(n * m), where n is the number of words in the file and m is the average length of the words. 

### Variation 3: Using Tries

In [ ]:
from rich.prompt import Prompt
from rich.console import Console
import random
import typing
import sys

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

SQUARES = {
    'correct_place': '🟩',
    'correct_letter': '🟨',
    'incorrect_letter': '⬛'
}

GAMEWORD_LIST_FNAME = "gamewords.txt"
GUESSWORD_LIST_FNAME = "guesswords.txt"

def filter_word(word: str, length: int) -> typing.Optional[str]:
    return word.strip().upper() if len(word.strip()) == length else None

def insert_word(root: TrieNode, word: str):
    node = root
    for char in word:
        if char not in node.children:
            node.children[char] = TrieNode()
        node = node.children[char]
    node.is_end_of_word = True

def create_word_trie(fname: str, length: int) -> TrieNode:
    root = TrieNode()
    with open(fname, "r") as f:
        lines = f.readlines()
    for line in lines:
        word = filter_word(line, length)
        if word:
            insert_word(root, word)
    return root

def get_random_word(root: TrieNode, length: int) -> str:
    node = root
    while True:
        word = ""
        while len(word) < length:
            # Choose a random child
            children_chars = list(node.children.keys())
            char = random.choice(children_chars)
            word += char
            node = node.children[char]
        if node.is_end_of_word:
            return word
        else:
            # If the word doesn't end here, backtrack and try again
            node = root

def validate(guess: str, wordlen: int, root: TrieNode) -> typing.Tuple[str, str]:
    guess_upper = guess.upper()
    if guess_upper == 'Q':
        raise KeyboardInterrupt
    node = root
    for char in guess_upper:
        if char not in node.children:
            return f"Guess must be a valid word", guess_upper
        node = node.children[char]
    if not node.is_end_of_word:
        return f"Guess must be of length {wordlen} and a valid word", guess_upper
    return None, guess_upper

def get_user_guess(wordlen: int, root: TrieNode) -> str:
    while True:
        guess = Prompt.ask("Guess:")
        error, guess = validate(guess=guess, wordlen=wordlen, root=root)
        if error is None:
            break
        print(error)
    return guess

def find_all_char_positions(word: str, char: str) -> typing.List[int]:
    positions = []
    pos = word.find(char)
    while pos != -1:
        positions.append(pos)
        pos = word.find(char, pos + 1)
    return positions

def compare(expected: str, guess: str) -> typing.List[str]:
    output = [SQUARES['incorrect_letter']] * len(expected)
    counted_pos = set()

    for index, (expected_char, guess_char) in enumerate(zip(expected, guess)):
        if expected_char == guess_char:
            output[index] = SQUARES['correct_place']
            counted_pos.add(index)

    for index, guess_char in enumerate(guess):
        if guess_char in expected and output[index] == SQUARES['incorrect_letter']:
            positions = find_all_char_positions(word=expected, char=guess_char)
            for pos in positions:
                if pos not in counted_pos:
                    output[index] = SQUARES['correct_letter']
                    counted_pos.add(pos)
                    break
    return output

def main():
    while True:
        WORDLEN = 5
        GAMEWORD_TRIE = create_word_trie(GAMEWORD_LIST_FNAME, length=WORDLEN)
        WORD = get_random_word(GAMEWORD_TRIE, WORDLEN)
        GAME_WORD_LENGTH = len(WORD)
        MAX_GUESSES = 6
        NUM_GUESSES = 0

        console = Console()
        console.print("""
        Guess words one at a time to guess the game word.
        
        🟩 character means a letter was guessed correctly
        in the correct position.
        🟨 character means a letter was guessed correctly,
        but in the incorrect position.
        ⬛ Incorrect letter.
        
        To quit, enter 'q'.
        """)

        console.print(" ".join(['⬛'] * GAME_WORD_LENGTH))

        try:
            while NUM_GUESSES < MAX_GUESSES:
                GUESS = get_user_guess(wordlen=GAME_WORD_LENGTH, root=GAMEWORD_TRIE)
                NUM_GUESSES += 1
                result = compare(expected=WORD, guess=GUESS)
                console.print(" ".join(result))

                if WORD == GUESS:
                    console.print(f"You won! It took you {NUM_GUESSES} guesses.")
                    break

            if NUM_GUESSES == MAX_GUESSES:
                console.print(f"Sorry, you reached the maximum number of guesses. The correct answer was {WORD.upper()}.")

        except KeyboardInterrupt:
            console.print(f"You quit - the correct answer was {WORD.upper()} and you took {NUM_GUESSES} guesses")

        play_again = Prompt.ask("Do you want to play again? (y/n):")
        if play_again.lower() != 'y':
            console.print("Thanks for playing!")
            break

if __name__ == '__main__':
    main()


Time Complexity:

create_word_trie and get_random_word: The time complexity of this function is O(n * m) because for each word, we iterate over its characters to insert them into the trie or randomly select a word from the trie, where n is the number of words in the file and m is the average word length.


get_user_guess: O(n*(m+m))~ O(n*m) n is the no of words in file and m is the average word length.

Space Complexity:

Trie data structure: The space complexity of the trie structure depends on the number of nodes it contains. In the worst case, each node may have 26 children, so the space complexity of the trie is O(n * m * 26).
